# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,827.668826,0.382777,1234,828.051603,827.286049,1656.103206,1654.572098
6,902.727689,0.264489,1234,902.992178,902.463200,1805.984356,1804.926401
7,367.573395,-0.514548,1234,367.058847,368.087943,734.117694,736.175887
8,380.201816,0.486200,1234,380.688016,379.715616,761.376033,759.431232
9,575.969457,0.769339,1234,576.738796,575.200118,1153.477591,1150.400235
10,67.720082,0.724987,1234,68.445069,66.995095,136.890138,133.990190
11,101.519086,-0.414839,1234,101.104248,101.933925,202.208495,203.867850
12,640.465183,0.995762,1234,641.460945,639.469422,1282.921890,1278.938843
13,788.374122,0.006341,1234,788.380464,788.367781,1576.760927,1576.735562
14,872.320115,-0.167144,1234,872.152971,872.487259,1744.305942,1744.974519


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-828.051603,827.286049
6,-902.992178,902.463200
7,-367.058847,368.087943
8,-380.688016,379.715616
9,-576.738796,575.200118
10,-68.445069,66.995095
11,-101.104248,101.933925
12,-641.460945,639.469422
13,-788.380464,788.367781
14,-872.152971,872.487259


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-828.051603,827.286049
6,-902.992178,902.463200
7,-367.058847,368.087943
8,-380.688016,379.715616
9,-576.738796,575.200118
10,-68.445069,66.995095
11,-101.104248,101.933925
12,-641.460945,639.469422
13,-788.380464,788.367781
14,-872.152971,872.487259


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-828.05160283,  827.28604895],
       [-902.99217803,  902.46320035],
       [-367.0588471 ,  368.08794339],
       [-380.68801648,  379.71561606],
       [-576.73879571,  575.2001176 ],
       [ -68.44506911,   66.99509521],
       [-101.10424768,  101.93392496],
       [-641.46094518,  639.46942171],
       [-788.38046373,  788.36778107],
       [-872.15297102,  872.4872593 ]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1654.103206,-1654.572098
6,1803.984356,-1804.926401
7,732.117694,-736.175887
8,759.376033,-759.431232
9,1151.477591,-1150.400235
10,134.890138,-133.990190
11,200.208495,-203.867850
12,1280.921890,-1278.938843
13,1574.760927,-1576.735562
14,1742.305942,-1744.974519


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
